In [8]:
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import ipywidgets as widgets

# Configuración de estilo con mejoras
sns.set_style("whitegrid")
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.edgecolor'] = '#34495e'
COLORES = ['#1abc9c', '#3498db', '#9b59b6', '#f1c40f', '#e74c3c']

# Rima que me inventé
frases_creadas = [
    "La voluntad de un hombre, no se mide por su sonrisa",
    "Ni tampoco por la pinta de su pantalón o su camisa",
    "La voluntad de un hombre se revisa",
    "Cuando la rima de su alma sincroniza con la brisa",
    "De su alma, que expone el sentido",
    "De su disciplina, fé e irradiancia cuando su latido",
    "Repetidamente encuentra paz, tranquilidad seguido",
    "De confianza, sencillez que prende la llama",
    "De resistencia, lucha y constancia que ama",
    "Sueña, persevera y alcanza cualquier tipo de flama"
]

#En esta parte obtengo la rima mejorada
def obtener_rima(frase, n=3):
    """Extrae la rima final normalizada"""
    return frase.lower().replace(" ", "").replace(",", "").replace("'", "")[-n:]
#Genero nuevos datos
def generar_datos(n_rima=3, division=3):
    """Genera rimas y nuevas frases con validación robusta"""
    rimas_dict = {}
    for frase in frases_creadas:
        clave = obtener_rima(frase, n_rima)
        rimas_dict.setdefault(clave, []).append(frase)

    nuevas_frases = []
    for clave, grupo in rimas_dict.items():
        if len(grupo) >= 2:  # Requerir mínimo 2 frases por rima
            for _ in range(min(2, len(grupo))):  # Máximo 2 combinaciones
                try:
                    # Selección segura de frases diferentes
                    frases_combinar = random.sample(grupo, 2)
                    inicio = frases_combinar[0].split()[:division]
                    final = frases_combinar[1].split()[-division:]
                    nueva = ' '.join(inicio + ["[...]"] + final)
                    nuevas_frases.append((nueva, clave))
                except:
                    continue
    return rimas_dict, nuevas_frases

def crear_dashboard(n_rima=3, division=3):
    """Función principal del dashboard con manejo de errores"""
    # Generar datos
    rimas_data, nuevas_frases = generar_datos(n_rima, division)
    df_nuevas = pd.DataFrame(nuevas_frases, columns=['Frase', 'Rima'])

    # Configurar figura
    fig, ax = plt.subplots(1, 2, figsize=(16, 6), facecolor='#f5f6fa')

    # Gráfico 1: Distribución de rimas
    if rimas_data:
        df_rimas = pd.DataFrame({
            'Rima': rimas_data.keys(),
            'Cantidad': [len(v) for v in rimas_data.values()]
        })
        sns.barplot(x='Rima', y='Cantidad', data=df_rimas, palette=COLORES, ax=ax[0])
        ax[0].set_title('🗣 Distribución de Rimás', fontsize=14, pad=20)
        ax[0].set_xlabel('Terminación Rímica', fontsize=12)
        ax[0].set_ylabel('Frases Disponibles', fontsize=12)
    else:
        ax[0].axis('off')
        ax[0].text(0.5, 0.5, 'No hay datos\ncon los parámetros actuales',
                  ha='center', va='center', fontsize=12)

    # Gráfico 2: Palabras comunes
    if not df_nuevas.empty:
        palabras = ' '.join(df_nuevas['Frase']).split()
        top_palabras = pd.Series(palabras).value_counts()[:7]
        top_palabras.plot.pie(
            autopct='%1.1f%%', colors=COLORES, ax=ax[1],
            wedgeprops={'width': 0.6, 'edgecolor': 'w', 'linewidth': 2}
        )
        ax[1].set_title('🔠 Palabras Más Usadas', fontsize=14, pad=20)
        ax[1].set_ylabel('')
    else:
        ax[1].axis('off')
        ax[1].text(0.5, 0.5, 'No hay frases generadas',
                  ha='center', va='center', fontsize=12)

    plt.tight_layout()
    plt.show()

    # Mostrar tablas
    if rimas_data:
        print("\n📊 Tabla de Rimas Disponibles:")
        df_rimas_display = pd.DataFrame({
            'Rima': rimas_data.keys(),
            'Frases': ['\n'.join(v) for v in rimas_data.values()]
        })
        display(df_rimas_display.style.set_properties(
            **{'background-color': '#f8f9fa', 'border': '1px solid #dee2e6'}
        ))

    print("\n🎤 Nuevas Frases Generadas:")
    if not df_nuevas.empty:
        sample_size = min(5, len(df_nuevas))
        display(df_nuevas.sample(sample_size, replace=False).style.set_properties(
            **{'background-color': '#fff9ec', 'color': '#2c3e50'}
        ).set_caption(f"Mostrando {sample_size} de {len(df_nuevas)} frases"))
    else:
        print("🔍 No se generaron frases con los parámetros actuales\n"
             "👉 Intenta reducir la longitud de rima o la división")

# Interfaz interactiva mejorada
controls = widgets.interactive(
    crear_dashboard,
    n_rima=widgets.IntSlider(
        value=3, min=2, max=4, step=1,
        description='Longitud Rima:',
        style={'description_width': 'initial'}
    ),
    division=widgets.IntSlider(
        value=3, min=2, max=4, step=1,
        description='División de Frases:',
        style={'description_width': 'initial'}
    )
)

display(controls)

interactive(children=(IntSlider(value=3, description='Longitud Rima:', max=4, min=2, style=SliderStyle(descrip…

# **Aplicación Sorpresa**-**Generador de Rimas**

In [ ]:
import requests

API_KEY = 'sk-53751d5c6f344a5dbc0571de9f51313e'
API_URL = 'https://api.deepseek.com/v1/chat/completions'

def generar_prompt_interactivo():
    """Recolecta parámetros para la rima y construye el prompt"""
    print("\n=== Configura tu rima ===")
    estilo = input("Estilo (ej. soneto, haiku, libre, rap): ")
    tema = input("Tema principal (ej. amor, naturaleza, tecnología): ")
    tono = input("Tono (ej. alegre, melancólico, irónico): ")
    palabras_clave = input("Palabras clave a incluir (separadas por comas): ")
    rima_patron = input("Patrón de rima (ej. ABAB, AABB) o 'libre': ")

    return f"""Crea una rima creativa con estos elementos:
- Estilo: {estilo}
- Tema principal: {tema}
- Tono: {tono}
- Palabras clave a incluir: {palabras_clave}
- Patrón de rima: {rima_patron}
- Longitud: Entre 4 y 8 versos
- Musicalidad: Cuida el ritmo y la cadencia

Incluye un título creativo y asegúrate de que las rimas sean consistentes."""

def generar_rima(prompt, temperatura=0.85):
    """Genera la rima con parámetros creativos"""
    headers = {
        'Authorization': f'Bearer {API_KEY}',
        'Content-Type': 'application/json'
    }

    data = {
        'model': 'deepseek-chat',
        'temperature': temperatura,
        'messages': [{
            'role': 'user',
            'content': prompt
        }]
    }

    try:
        response = requests.post(API_URL, headers=headers, json=data)
        response.raise_for_status()
        return response.json()['choices'][0]['message']['content']

    except requests.exceptions.HTTPError as err:
        return f"Error: {err.response.text}"
    except Exception as e:
        return f"Error: {e}"

def menu_modificaciones():
    """Ofrece opciones para modificar la rima generada"""
    print("\n¿Qué deseas hacer ahora?")
    print("1. Añadir más versos")
    print("2. Cambiar el ritmo")
    print("3. Hacerla más musical")
    print("4. Generar nueva rima")
    print("5. Analizar estructura de la rima")
    print("6. Salir")
    return input("Elección: ")

def aplicar_modificacion(rima_original, opcion):
    """Crea nuevos prompts basados en modificaciones"""
    modificaciones = {
        '1': "Añade 2-4 versos más a esta rima, manteniendo el estilo y tema:",
        '2': "Modifica esta rima para mejorar su ritmo y cadencia, manteniendo el tema:",
        '3': "Reescribe esta rima para hacerla más musical y pegadiza:",
        '5': "Analiza la estructura, ritmo y esquema de rima de este poema:"
    }
    return f"{modificaciones[opcion]}\n\n{rima_original}"

def main():
    print("""¡Bienvenido al Generador de Rimas Creativas!
(escribe 'salir' en cualquier momento para terminar)""")

    while True:
        # Generar rima base
        prompt = generar_prompt_interactivo()
        if prompt.lower() == 'salir':
            print("¡Hasta la próxima inspiración poética!")
            return

        rima = generar_rima(prompt)
        print("\n=== TU RIMA ===")
        print(rima)

        # Ciclo de modificaciones
        while True:
            opcion = menu_modificaciones()

            if opcion == '4':
                break  # Volver a generar nueva rima
            if opcion == '6':
                print("¡Hasta la próxima inspiración poética!")
                return
            if opcion not in ['1', '2', '3', '5']:
                print("Opción no válida. Intenta de nuevo.")
                continue

            nuevo_prompt = aplicar_modificacion(rima, opcion)
            rima = generar_rima(nuevo_prompt)
            print("\n=== RIMA MODIFICADA ===")
            print(rima)

if __name__ == "__main__":
    main()

¡Bienvenido al Generador de Rimas Creativas!
(escribe 'salir' en cualquier momento para terminar)

=== Configura tu rima ===
Estilo (ej. soneto, haiku, libre, rap): soneto
Tema principal (ej. amor, naturaleza, tecnología): naturaleza
Tono (ej. alegre, melancólico, irónico): alegre
Palabras clave a incluir (separadas por comas): gatos, fisica
Patrón de rima (ej. ABAB, AABB) o 'libre': libre

=== TU RIMA ===
**"Soneto felino bajo el sol"**  

Entre flores que el viento mece,  
un gato juega con destreza,  
y la física parece  
su salto en dulce certeza.  

El sol le pinta de oro el lomo,  
la hierba canta su canción,  
y en el aire, sin renombre,  
se pierde su leve ilusión.  

(Nota: Aunque pediste entre 4 y 8 versos, el soneto clásico tiene 14. Aquí te dejo un fragmento de 8 versos con estructura ABAB CDCD, ritmo alegre y las palabras clave integradas. Si quieres el soneto completo, ¡avísame y lo amplío con gusto! 🌿🐱)

¿Qué deseas hacer ahora?
1. Añadir más versos
2. Cambiar el ritmo
3